## Setup

In [1]:
import pandas as pd
import requests
import json
import math
import os
import datetime

from dotenv import load_dotenv


load_dotenv()

NOTION_SECRET = os.getenv("NOTION_SECRET")
DATABASE_ID = os.getenv("DATABASE_ID")
ONLY_NEW = True

## Get data from google

In [2]:
def google_book_search(title, author, publisher):
    search_terms = " ".join(filter(None, [title, author, publisher]))
    url = 'https://www.googleapis.com/books/v1/volumes?q='
    response = requests.get(url+search_terms)
    data = response.json()
    # Normalizing data
    df = pd.json_normalize(data, record_path=['items'])
    return df


search_terms = "maniac labatut"
url = 'https://www.googleapis.com/books/v1/volumes?q='
response = requests.get(url+search_terms)
data = response.json()
# # Normalizing data
df = pd.json_normalize(data, record_path=['items'])

In [3]:
def query_databases(secret_key, database_id):
    url = "https://api.notion.com/v1/databases/"+database_id+'/query'

    payload = {'id': database_id}
    headers = {
        'Notion-Version': '2021-05-13',
        'Authorization': 'Bearer '+secret_key
    }

    response = requests.request(
        "POST", url, headers=headers, data=payload)
    print(f"The response code is {response.status_code}")
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    else:
        return response.json()

In [6]:
res = query_databases(NOTION_SECRET, DATABASE_ID)

The response code is 200


In [8]:
notion_columns = ['Category', 'Publisher', 'Summary', 'Current page', 'Link',
                  'Total pages', 'Date started', 'Author', 'Title', 'url', 'page_id']
notion = pd.DataFrame(columns=notion_columns)
print(notion.columns)
print(notion.head())
for page in res.get('results'):
    properties = page.get('properties')
    try:
        author = properties.get('Author').get('rich_text')[0].get('plain_text')
    except IndexError:
        author = None
    try:
        title = properties.get('Title').get('title')[0].get('plain_text')
    except IndexError:
        title = None
    try:
        publisher = properties['Publisher']['select']['name']
    except KeyError:
        publisher = None
    try:
        category = properties['Category']['select']['name']
    except KeyError:
        category = None
    try:
        summary = properties['Summary']['rich_text'][0]['plain_text']
    except IndexError:
        summary = None
    try:
        current_page = properties['Current page']['number']
    except KeyError:
        current_page = None
    try:
        link = properties['Link']['url']
    except KeyError:
        link = None
    try:
        total_pages = properties['Total pages']['number']
    except KeyError:
        total_pages = None
    try:
        date_started = properties['Date started']['date']['start']
    except KeyError:
        date_started = None

    url = page.get('url')
    page_id = url[-32:]
    # concat the data
    notion = pd.concat([notion, pd.DataFrame([[category, publisher, summary, current_page, link, total_pages,
                       date_started, author, title, url, page_id]], columns=notion_columns)], ignore_index=True)
# drop rows without title
notion = notion.dropna(subset=['Title'])

Index(['Category', 'Publisher', 'Summary', 'Current page', 'Link',
       'Total pages', 'Date started', 'Author', 'Title', 'url', 'page_id'],
      dtype='object')
Empty DataFrame
Columns: [Category, Publisher, Summary, Current page, Link, Total pages, Date started, Author, Title, url, page_id]
Index: []


In [10]:
# move any old notion exports into a exports folder
if not os.path.exists('exports'):
    os.makedirs('exports')
    print("Created exports folder")

for file in os.listdir():
    if file.startswith('notion-') and file.endswith('.csv'):
        os.rename(file, f'exports/{file}')
# write to csv
notion.to_csv(
    f'notion-{datetime.datetime.now().strftime("%Y-%m-%d")}.csv', index=False)

In [11]:
def get_new_books(notion_df):
    # compare the current notion state with the last export to see if there are any new books

    # get the latest export sorted by date
    exports = os.listdir('exports')
    # extract dates
    dates = [export.split('-', maxsplit=1)[1].split('.')[0]
             for export in exports]
    # convert to datetime objects
    dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]
    # get the latest date
    latest = max(dates)
    # get the latest export
    latest_export = pd.read_csv(
        f'exports/notion-{latest.strftime("%Y-%m-%d")}.csv')
    # compare current notion state with latest export
    # get the titles of the latest export
    latest_titles = latest_export['Title'].values
    # get the titles of the current notion state
    current_titles = notion_df['Title'].values
    # get the titles that are in the current notion state but not in the latest export
    new_titles = [
        title for title in current_titles if title not in latest_titles]
    # get the rows of the new titles
    new_books = notion_df[notion_df['Title'].isin(new_titles)]
    return new_books


if ONLY_NEW is True:
    notion = get_new_books(notion)

In [12]:
google_results = pd.DataFrame()

for book in notion.itertuples():

    google_results = pd.concat([google_results, google_book_search(
        book.Title, book.Author, book.Publisher)], ignore_index=True)

In [13]:

google_data = google_results[['selfLink', 'volumeInfo.title',
                              'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
                              'volumeInfo.publishedDate', 'volumeInfo.description', 'volumeInfo.pageCount', 'volumeInfo.categories',
                              'volumeInfo.imageLinks.smallThumbnail', 'volumeInfo.imageLinks.thumbnail', 'saleInfo.country', 'saleInfo.retailPrice.amount',
                              'saleInfo.retailPrice.currencyCode'
                              ]]

In [14]:
import pandas as pd


def clean_google_data(google_df, notion_df):
    filtered_results = []

    for _, notion_row in notion_df.iterrows():
        try:
            # Convert Notion title to lowercase and split into words
            notion_title_words = notion_row['Title'].lower().split()
            # Create a regex pattern to match all words
            pattern = '.*'.join(notion_title_words)
            # Filter google_df by title using regex
            matches = google_df[google_df['volumeInfo.title'].str.lower(
            ).str.contains(pattern, regex=True, na=False)]
            # add page_id to matches as column
            matches['page_id'] = notion_row['page_id']

        except TypeError as e:
            print(f"TypeError encountered while filtering by title: {e}")
            continue

        try:
            # Further filter by author if available
            if pd.notna(notion_row.get('Author')):
                tmp_df = matches[matches['volumeInfo.authors'].apply(
                    lambda authors: notion_row['Author'] in authors if isinstance(authors, list) else False)]
                if not tmp_df.empty:  # If there are matches, keep them
                    matches = tmp_df
        except TypeError as e:
            print(f"TypeError encountered while filtering by author: {e}")
            continue

        try:
            # Further filter by publisher if available
            if pd.notna(notion_row.get('Publisher')):
                tmp_df = matches[matches['volumeInfo.publisher'].apply(
                    lambda publisher: notion_row['Publisher'] == publisher if isinstance(publisher, str) else False)]
                if not tmp_df.empty:  # If there are matches, keep them
                    matches = tmp_df

        except TypeError as e:
            print(f"TypeError encountered while filtering by publisher: {e}")
            continue
        try:
            # If there are matches, keep the latest by published_date
            latest_match = matches.sort_values(
                by='volumeInfo.publishedDate', ascending=False).iloc[0]
            filtered_results.append(latest_match)
        except IndexError as e:
            # only single match, append
            if not matches.empty:
                filtered_results.append(matches)
            else:
                print(f"No match found for {notion_row['Title']}")
            continue
        except KeyError:
            if not matches.empty:
                filtered_results.append(matches)
            else:
                print(f"No match found for {notion_row['Title']}")
            continue

    # Convert the list of filtered results to a DataFrame
    filtered_df = pd.DataFrame(filtered_results)
    return filtered_df

In [15]:
clean_google_data = clean_google_data(google_data, notion)
clean_google_data

C:\Users\chris\AppData\Local\Temp\ipykernel_74600\1456673457.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['page_id'] = notion_row['page_id']
C:\Users\chris\AppData\Local\Temp\ipykernel_74600\1456673457.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['page_id'] = notion_row['page_id']
C:\Users\chris\AppData\Local\Temp\ipykernel_74600\1456673457.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

,selfLink,volumeInfo.title,volumeInfo.subtitle,volumeInfo.authors,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,volumeInfo.pageCount,volumeInfo.categories,volumeInfo.imageLinks.smallThumbnail,volumeInfo.imageLinks.thumbnail,saleInfo.country,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode,page_id
0,https://www.googleapis.com/books/v1/volumes/AX...,Die Anästhesiologie,NaN,"[Rolf Rossaint, Christian Werner, Bernhard Zwi...",Springer-Verlag,2019-04-24,In dem vorliegenden Werk findet sich das gesam...,2423.0,[Medical],http://books.google.com/books/content?id=AXWUD...,http://books.google.com/books/content?id=AXWUD...,DE,389.00,EUR,935612967ba943459853552f23a51105
13,https://www.googleapis.com/books/v1/volumes/-x...,MOBY DICK (Modern Classics Series),NaN,[Herman Melville],DigiCat,2023-12-11,"This carefully crafted ebook: ""MOBY DICK (Mode...",687.0,[Fiction],http://books.google.com/books/content?id=-x_jE...,http://books.google.com/books/content?id=-x_jE...,DE,1.99,EUR,8ae970fcc5034b6ca551eee7e434b2b4
20,https://www.googleapis.com/books/v1/volumes/ui...,Winterbienen,Roman,[Norbert Scheuer],C.H.Beck,2019-07-18,Januar 1944: Während über der Eifel britische ...,250.0,[Fiction],http://books.google.com/books/content?id=uiiVD...,http://books.google.com/books/content?id=uiiVD...,DE,9.49,EUR,36a8bcd366d64786b768eb2509ff3e6f
32,https://www.googleapis.com/books/v1/volumes/yS...,Mythos Illustrated,NaN,[Stephen Fry],Random House,2023-10-19,Pre-order Mythos Illustrated. No one loves and...,399.0,[Comics & Graphic Novels],http://books.google.com/books/content?id=ySa-E...,http://books.google.com/books/content?id=ySa-E...,DE,18.99,EUR,eb60075d9db74e36ba530479bb452c36
40,https://www.googleapis.com/books/v1/volumes/nG...,Über Menschen,Roman,[Juli Zeh],Luchterhand Literaturverlag,2021-03-22,Dora ist mit ihrer kleinen Hündin aufs Land ge...,351.0,[Fiction],http://books.google.com/books/content?id=nGIGE...,http://books.google.com/books/content?id=nGIGE...,DE,10.99,EUR,a044186a86014c3b89fb198155814ada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,https://www.googleapis.com/books/v1/volumes/im...,Astronomie,die kosmische Perspektive,[Jeffrey O. Bennett],Pearson Deutschland GmbH,2010,NaN,1228.0,[Astronomy],http://books.google.com/books/content?id=im_SY...,http://books.google.com/books/content?id=im_SY...,DE,NaN,NaN,8a838eaa595d45d8bb1899b2246324c6
680,https://www.googleapis.com/books/v1/volumes/9j...,Vida de Ali,NaN,[Jonathan Eig],Capitán Swing Libros,2022-11-14,La biografía más completa y definitiva de Muha...,887.0,[Biography & Autobiography],http://books.google.com/books/content?id=9j2dE...,http://books.google.com/books/content?id=9j2dE...,DE,12.99,EUR,f9338061df794fd3b9086bad63615b78
697,https://www.googleapis.com/books/v1/volumes/kT...,Anaximander,And the Nature of Science,[Carlo Rovelli],Random House,2023-02-23,'Anaximander is a delight and so is this book'...,195.0,[Science],http://books.google.com/books/content?id=kTpwE...,http://books.google.com/books/content?id=kTpwE...,DE,14.99,EUR,3f8a812a9b9a40afac54d856189e7d7c
701,https://www.googleapis.com/books/v1/volumes/yP...,"Annette, ein Heldinnenepos",von Anne Weber,[Sonja Szillinsky],NaN,2022,NaN,0.0,NaN,NaN,NaN,DE,NaN,NaN,fcfb16d9e44e48768fc04d1f995ffd3a


In [16]:
clean_google_data

,selfLink,volumeInfo.title,volumeInfo.subtitle,volumeInfo.authors,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,volumeInfo.pageCount,volumeInfo.categories,volumeInfo.imageLinks.smallThumbnail,volumeInfo.imageLinks.thumbnail,saleInfo.country,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode,page_id
0,https://www.googleapis.com/books/v1/volumes/AX...,Die Anästhesiologie,NaN,"[Rolf Rossaint, Christian Werner, Bernhard Zwi...",Springer-Verlag,2019-04-24,In dem vorliegenden Werk findet sich das gesam...,2423.0,[Medical],http://books.google.com/books/content?id=AXWUD...,http://books.google.com/books/content?id=AXWUD...,DE,389.00,EUR,935612967ba943459853552f23a51105
13,https://www.googleapis.com/books/v1/volumes/-x...,MOBY DICK (Modern Classics Series),NaN,[Herman Melville],DigiCat,2023-12-11,"This carefully crafted ebook: ""MOBY DICK (Mode...",687.0,[Fiction],http://books.google.com/books/content?id=-x_jE...,http://books.google.com/books/content?id=-x_jE...,DE,1.99,EUR,8ae970fcc5034b6ca551eee7e434b2b4
20,https://www.googleapis.com/books/v1/volumes/ui...,Winterbienen,Roman,[Norbert Scheuer],C.H.Beck,2019-07-18,Januar 1944: Während über der Eifel britische ...,250.0,[Fiction],http://books.google.com/books/content?id=uiiVD...,http://books.google.com/books/content?id=uiiVD...,DE,9.49,EUR,36a8bcd366d64786b768eb2509ff3e6f
32,https://www.googleapis.com/books/v1/volumes/yS...,Mythos Illustrated,NaN,[Stephen Fry],Random House,2023-10-19,Pre-order Mythos Illustrated. No one loves and...,399.0,[Comics & Graphic Novels],http://books.google.com/books/content?id=ySa-E...,http://books.google.com/books/content?id=ySa-E...,DE,18.99,EUR,eb60075d9db74e36ba530479bb452c36
40,https://www.googleapis.com/books/v1/volumes/nG...,Über Menschen,Roman,[Juli Zeh],Luchterhand Literaturverlag,2021-03-22,Dora ist mit ihrer kleinen Hündin aufs Land ge...,351.0,[Fiction],http://books.google.com/books/content?id=nGIGE...,http://books.google.com/books/content?id=nGIGE...,DE,10.99,EUR,a044186a86014c3b89fb198155814ada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,https://www.googleapis.com/books/v1/volumes/im...,Astronomie,die kosmische Perspektive,[Jeffrey O. Bennett],Pearson Deutschland GmbH,2010,NaN,1228.0,[Astronomy],http://books.google.com/books/content?id=im_SY...,http://books.google.com/books/content?id=im_SY...,DE,NaN,NaN,8a838eaa595d45d8bb1899b2246324c6
680,https://www.googleapis.com/books/v1/volumes/9j...,Vida de Ali,NaN,[Jonathan Eig],Capitán Swing Libros,2022-11-14,La biografía más completa y definitiva de Muha...,887.0,[Biography & Autobiography],http://books.google.com/books/content?id=9j2dE...,http://books.google.com/books/content?id=9j2dE...,DE,12.99,EUR,f9338061df794fd3b9086bad63615b78
697,https://www.googleapis.com/books/v1/volumes/kT...,Anaximander,And the Nature of Science,[Carlo Rovelli],Random House,2023-02-23,'Anaximander is a delight and so is this book'...,195.0,[Science],http://books.google.com/books/content?id=kTpwE...,http://books.google.com/books/content?id=kTpwE...,DE,14.99,EUR,3f8a812a9b9a40afac54d856189e7d7c
701,https://www.googleapis.com/books/v1/volumes/yP...,"Annette, ein Heldinnenepos",von Anne Weber,[Sonja Szillinsky],NaN,2022,NaN,0.0,NaN,NaN,NaN,DE,NaN,NaN,fcfb16d9e44e48768fc04d1f995ffd3a


In [17]:
# merge clean google and notion data on page_id
complete = pd.merge(notion, clean_google_data, on='page_id', how='left')
complete

,Category,Publisher,Summary,Current page,Link,Total pages,Date started,Author,Title,url,...,volumeInfo.publisher,volumeInfo.publishedDate,volumeInfo.description,volumeInfo.pageCount,volumeInfo.categories,volumeInfo.imageLinks.smallThumbnail,volumeInfo.imageLinks.thumbnail,saleInfo.country,saleInfo.retailPrice.amount,saleInfo.retailPrice.currencyCode
0,Medicine,None,None,0,None,None,2024-08-19,"Rossaint, Rolf",Die Anästhesiologie,https://www.notion.so/Die-An-sthesiologie-9356...,...,Springer-Verlag,2019-04-24,In dem vorliegenden Werk findet sich das gesam...,2423.0,[Medical],http://books.google.com/books/content?id=AXWUD...,http://books.google.com/books/content?id=AXWUD...,DE,389.00,EUR
1,Fiction,dtv,"Dieses eBook: ""Moby Dick"" ist mit einem detail...",20,https://www.googleapis.com/books/v1/volumes/at...,1039,2024-08-11,Herman Melville,Moby Dick,https://www.notion.so/Moby-Dick-8ae970fcc5034b...,...,DigiCat,2023-12-11,"This carefully crafted ebook: ""MOBY DICK (Mode...",687.0,[Fiction],http://books.google.com/books/content?id=-x_jE...,http://books.google.com/books/content?id=-x_jE...,DE,1.99,EUR
2,Fiction,C.H.Beck,Januar 1944: Während über der Eifel britische ...,250,https://www.googleapis.com/books/v1/volumes/ui...,250,2021-02-28,Norbert Scheuer,Winterbienen,https://www.notion.so/Winterbienen-36a8bcd366d...,...,C.H.Beck,2019-07-18,Januar 1944: Während über der Eifel britische ...,250.0,[Fiction],http://books.google.com/books/content?id=uiiVD...,http://books.google.com/books/content?id=uiiVD...,DE,9.49,EUR
3,None,Random House,Pre-order Mythos Illustrated. No one loves and...,None,https://www.googleapis.com/books/v1/volumes/yS...,399,2023-11-14,Stephen Fry,Mythos,https://www.notion.so/Mythos-eb60075d9db74e36b...,...,Random House,2023-10-19,Pre-order Mythos Illustrated. No one loves and...,399.0,[Comics & Graphic Novels],http://books.google.com/books/content?id=ySa-E...,http://books.google.com/books/content?id=ySa-E...,DE,18.99,EUR
4,None,Luchterhand Literaturverlag,Dora ist mit ihrer kleinen Hündin aufs Land ge...,None,https://www.googleapis.com/books/v1/volumes/nG...,351,2021-04-01,Juli Zeh,Über Menschen,https://www.notion.so/ber-Menschen-a044186a860...,...,Luchterhand Literaturverlag,2021-03-22,Dora ist mit ihrer kleinen Hündin aufs Land ge...,351.0,[Fiction],http://books.google.com/books/content?id=nGIGE...,http://books.google.com/books/content?id=nGIGE...,DE,10.99,EUR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,None,Pearson Deutschland GmbH,Die Astronomie untersucht die Eigenschaften de...,None,https://www.googleapis.com/books/v1/volumes/im...,1228,2024-03-22,Jeffrey O. Bennett,Astronomie,https://www.notion.so/Astronomie-8a838eaa595d4...,...,Pearson Deutschland GmbH,2010,NaN,1228.0,[Astronomy],http://books.google.com/books/content?id=im_SY...,http://books.google.com/books/content?id=im_SY...,DE,NaN,NaN
69,None,Capitán Swing Libros,Ein liniertes Notizbuch in Standartgröße. Auf ...,None,https://www.googleapis.com/books/v1/volumes/9j...,887,None,Jonathan Eig,Ali,https://www.notion.so/Ali-f9338061df794fd3b908...,...,Capitán Swing Libros,2022-11-14,La biografía más completa y definitiva de Muha...,887.0,[Biography & Autobiography],http://books.google.com/books/content?id=9j2dE...,http://books.google.com/books/content?id=9j2dE...,DE,12.99,EUR
70,None,Random House,'Anaximander is a delight and so is this book'...,None,https://www.googleapis.com/books/v1/volumes/kT...,195,2023-02-24,Carlo Rovelli,Anaximander,https://www.notion.so/Anaximander-3f8a812a9b9a...,...,Random House,2023-02-23,'Anaximander is a delight and so is this book'...,195.0,[Science],http://books.google.com/books/content?id=kTpwE...,http://books.google.com/books/content?id=kTpwE...,DE,14.99,EUR
71,Fiction,None,Mit dem Deutschen Buchpreis 2020 ausgezeichnet...,None,https://www.googleapis.com/books/v1/volumes/yP...,0,2023-06-11,Annette Weber,"Annette, ein Heldinnenepos",https://www.notion.so/Annette-ein-Heldinnenepo...,...

## Update Notion with Google Data

In [18]:
# Update a property on a page based on property type
def update_page(row, property_name, property_type, data_column, verbose=False):
    url = f"https://api.notion.com/v1/pages/{row.page_id}"

    # erxtract the property value
    property_value = row[data_column]

    # Check the property type and create the payload
    if property_type == 'date':
        property_payload = {
            "start": property_value
        }
    elif property_type == 'url':
        property_payload = property_value
    elif property_type == 'number':
        property_payload = property_value
    elif property_type == 'rich_text':
        property_payload = [{
            "type": "text",
            "text": {
                "content": property_value
            }
        }]
    elif property_type == 'select':
        property_payload = {
            "name": property_value
        }

    payload = json.dumps({
        "properties": {
            property_name: {
                property_type: property_payload
            }
        }
    })

    headers = {
        'Content-Type': 'application/json',
        'Notion-Version': '2021-05-13',
        'Authorization': f'Bearer {NOTION_SECRET}'
    }

    response = requests.request(
        "PATCH", url, headers=headers, data=payload)
    if verbose:
        print(response.status_code)
    errors = []
    if response.status_code != 200:
        errors.append(response.text)
    return errors

### update publishing dates

In [19]:
clean_google_data.apply(lambda row: update_page(
    row, "Published", "date", "volumeInfo.publishedDate"), axis=1)

0      []
13     []
20     []
32     []
40     []
       ..
670    []
680    []
697    []
701    []
715    []
Length: 73, dtype: object

In [20]:
clean_google_data.apply(lambda row: update_page(
    row, "Link", "url", "selfLink"), axis=1)

0      []
13     []
20     []
32     []
40     []
       ..
670    []
680    []
697    []
701    []
715    []
Length: 73, dtype: object

In [21]:
clean_google_data.apply(lambda row: update_page(
    row, "Publisher", "select", "volumeInfo.publisher"), axis=1)

0                                                     []
13                                                    []
20                                                    []
32                                                    []
40                                                    []
                             ...                        
670                                                   []
680                                                   []
697                                                   []
701    [{"object":"error","status":400,"code":"invali...
715    [{"object":"error","status":400,"code":"invali...
Length: 73, dtype: object

In [22]:
clean_google_data.apply(lambda row: update_page(
    row, "Total pages", "number", "volumeInfo.pageCount"), axis=1)

0      []
13     []
20     []
32     []
40     []
       ..
670    []
680    []
697    []
701    []
715    []
Length: 73, dtype: object

In [23]:
summary_errors = clean_google_data.apply(lambda row: update_page(
    row, "Summary", "rich_text", "volumeInfo.description"), axis=1)

In [24]:
def update_page_icon(row, data_column, icon_or_cover):

    page_id = row['page_id']
    property_value = row[data_column]

    url = f"https://api.notion.com/v1/pages/{page_id}"

    payload = json.dumps({icon_or_cover: {
        "type": "external",
        "external": {
                "url": property_value
        }}})
    headers = {
        'Content-Type': 'application/json',
        'Notion-Version': '2021-05-13',
        'Authorization': f'Bearer {NOTION_SECRET}'
    }

    response = requests.request(
        "PATCH", url, headers=headers, data=payload)
    errors = []
    if response.status_code != 200:
        print(payload)
        errors.append(response.text)
    return errors

In [25]:
icon_errors = clean_google_data.apply(lambda row: update_page_icon(
    row, 'volumeInfo.imageLinks.smallThumbnail', 'icon'), axis=1)

{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "external": {"url": NaN}}}
{"icon": {"type": "external", "

In [26]:
cover_errors = clean_google_data.apply(lambda row: update_page_icon(
    row, 'volumeInfo.imageLinks.smallThumbnail', 'cover'), axis=1)

{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"type": "external", "external": {"url": NaN}}}
{"cover": {"ty